<a href="https://colab.research.google.com/github/iterat1on/class/blob/main/pyspark_%EB%9E%9C%EB%8D%A4%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8%2C_%EA%B2%B0%EC%A0%95%ED%8A%B8%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=91cd33db5e79588191d2ec7d0699ee9deb2628294329355f302240fcb7c950bd
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [3]:
import pyspark as spark
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
import glob
from pyspark import SparkContext
from pyspark.sql import Row
from pyspark.sql import SQLContext

sc = SparkContext()
sqlContext = SQLContext(sc)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
df = pd.read_csv('/content/final_data.csv',encoding='utf-8')
df

,현장안전센터거리,시간단위기온,시간단위풍속,시간단위가시거리,lon,lat,실효습도,가스사용량(KWh),전기사용량(KWh),화재발생시각,인명피해확률,회귀예측,지수,등급,인덱스
0,1.0,6.3,2.3,294,127.089946,37.583679,57.376250,3519.916667,3519.916667,주간,2.747923,82.710463,15.075875,3등급,1
1,2.0,7.6,2.1,409,127.027563,37.499615,57.376250,41868.750000,41868.750000,주간,2.105026,84.887385,13.367504,3등급,2
2,4.0,9.2,3.1,331,127.016532,37.514922,57.376250,6635.750000,6635.750000,주간,1.817727,87.123438,12.584382,3등급,3
3,2.0,8.3,3.6,305,127.083044,37.647071,57.376250,457670.916700,457670.916700,주간,0.120060,96.967139,3.412013,3등급,4
4,4.0,6.4,2.1,221,127.077849,37.534442,57.376250,4711.583333,4711.583333,저녁,1.155019,85.483158,9.936530,3등급,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24350,2.3,1.7,1.7,1404,126.948889,37.482005,52.885305,20237.500000,20237.500000,주간,4.354363,81.321135,18.817590,3등급,24351
24351,1.2,-0.8,1.4,1171,127.085319,37.503992,52.885305,14469.333330,14469.333330,저녁,3.407635,77.631818,16.264714,3등급,24352
24352,1.0,-0.8,1.4,1171,127.129703,37.542661,52.885305,1892.000000,1892.000000,저녁,3.723698,76.866137,16.918223,3등급,24353
24353,1.2,-1.0,0.9,1134,126.921296,37.552125,52.885305,9634.000000,9634.000000,저녁,3.287761,77.971913,16.011028,3등급,24354


In [5]:
def convert_Int(x) :
  try :
    result = int(x[0])
  except ValueError :
    pass

  return result


In [6]:
df["등급"] = df.등급.apply(lambda x : convert_Int(x))
df

,현장안전센터거리,시간단위기온,시간단위풍속,시간단위가시거리,lon,lat,실효습도,가스사용량(KWh),전기사용량(KWh),화재발생시각,인명피해확률,회귀예측,지수,등급,인덱스
0,1.0,6.3,2.3,294,127.089946,37.583679,57.376250,3519.916667,3519.916667,주간,2.747923,82.710463,15.075875,3,1
1,2.0,7.6,2.1,409,127.027563,37.499615,57.376250,41868.750000,41868.750000,주간,2.105026,84.887385,13.367504,3,2
2,4.0,9.2,3.1,331,127.016532,37.514922,57.376250,6635.750000,6635.750000,주간,1.817727,87.123438,12.584382,3,3
3,2.0,8.3,3.6,305,127.083044,37.647071,57.376250,457670.916700,457670.916700,주간,0.120060,96.967139,3.412013,3,4
4,4.0,6.4,2.1,221,127.077849,37.534442,57.376250,4711.583333,4711.583333,저녁,1.155019,85.483158,9.936530,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24350,2.3,1.7,1.7,1404,126.948889,37.482005,52.885305,20237.500000,20237.500000,주간,4.354363,81.321135,18.817590,3,24351
24351,1.2,-0.8,1.4,1171,127.085319,37.503992,52.885305,14469.333330,14469.333330,저녁,3.407635,77.631818,16.264714,3,24352
24352,1.0,-0.8,1.4,1171,127.129703,37.542661,52.885305,1892.000000,1892.000000,저녁,3.723698,76.866137,16.918223,3,24353
24353,1.2,-1.0,0.9,1134,126.921296,37.552125,52.885305,9634.000000,9634.000000,저녁,3.287761,77.971913,16.011028,3,24354


In [7]:
spark_df = sqlContext.createDataFrame(df)

In [14]:
df = pd.get_dummies(df, columns = ['화재발생시각'] )

In [15]:
spark_df = sqlContext.createDataFrame(df)

In [16]:
from pyspark.ml.feature import VectorAssembler

# 1. Train/Test 데이터로 분할
train_df, test_df = spark_df.randomSplit(weights=[0.7, 0.3], seed=123)

# 2. Feature Vector로 만들기
ftr_columns = ['현장안전센터거리', '시간단위기온', '시간단위풍속', '시간단위가시거리', '실효습도',
               '전기사용량(KWh)', '가스사용량(KWh)', '화재발생시각_새벽', '화재발생시각_저녁', '화재발생시각_주간']
vec_assembler = VectorAssembler(inputCols=ftr_columns, outputCol='features') #features에는 하나의 칼럼.

train_ftr_vec = vec_assembler.transform(train_df)
test_ftr_vec = vec_assembler.transform(test_df)  #테스트 데이터에 대해도 동일한 객체 사용
sparktrain_ftr_vec = vec_assembler.transform(test_df)

print('# 학습 원본 데이터:')
train_df.limit(5).show()

print('# Feature Vectorization 후 학습 데이터:')
train_ftr_vec.limit(5).show()

# 학습 원본 데이터:
+----------------+------------+------------+----------------+-----------+----------+---------+---------------+---------------+------------+-----------+-----------+----+------+---------------+-----------------+-----------------+-----------------+
|현장안전센터거리|시간단위기온|시간단위풍속|시간단위가시거리|        lon|       lat| 실효습도|가스사용량(KWh)|전기사용량(KWh)|인명피해확률|   회귀예측|       지수|등급|인덱스|화재발생시각_밤|화재발생시각_새벽|화재발생시각_저녁|화재발생시각_주간|
+----------------+------------+------------+----------------+-----------+----------+---------+---------------+---------------+------------+-----------+-----------+----+------+---------------+-----------------+-----------------+-----------------+
|             0.0|       -14.6|         1.8|            1746|127.0178003|37.5650556|36.283587|         5925.0|         5925.0|  82.0347788| 13.7803916| 33.6224832|   2|  4734|              1|                0|                0|                0|
|             0.0|       -14.2|         2.2|            2000|126.9386158|37.5111579| 33.27066

# Random_Forest

Test Accuracy : 90.1%

In [17]:
from pyspark.ml.classification import RandomForestClassifier

dt_clf = RandomForestClassifier(featuresCol='features', labelCol='등급', numTrees=2)

dt_model = dt_clf.fit(train_ftr_vec)

train_pred = dt_model.transform(train_ftr_vec)
test_pred = dt_model.transform(test_ftr_vec)

In [18]:
train_pred.show()

+----------------+------------+------------+----------------+-----------+----------+---------+---------------+---------------+------------+-----------+-----------+----+------+---------------+-----------------+-----------------+-----------------+--------------------+--------------------+--------------------+----------+
|현장안전센터거리|시간단위기온|시간단위풍속|시간단위가시거리|        lon|       lat| 실효습도|가스사용량(KWh)|전기사용량(KWh)|인명피해확률|   회귀예측|       지수|등급|인덱스|화재발생시각_밤|화재발생시각_새벽|화재발생시각_저녁|화재발생시각_주간|            features|       rawPrediction|         probability|prediction|
+----------------+------------+------------+----------------+-----------+----------+---------+---------------+---------------+------------+-----------+-----------+----+------+---------------+-----------------+-----------------+-----------------+--------------------+--------------------+--------------------+----------+
|             0.0|       -14.6|         1.8|            1746|127.0178003|37.5650556|36.283587|         5925.0|         5925.0|  82

In [19]:
test_pred.show()

+----------------+------------+------------+----------------+-----------+----------+---------+---------------+---------------+------------+-----------+-----------+----+------+---------------+-----------------+-----------------+-----------------+--------------------+--------------------+--------------------+----------+
|현장안전센터거리|시간단위기온|시간단위풍속|시간단위가시거리|        lon|       lat| 실효습도|가스사용량(KWh)|전기사용량(KWh)|인명피해확률|   회귀예측|       지수|등급|인덱스|화재발생시각_밤|화재발생시각_새벽|화재발생시각_저녁|화재발생시각_주간|            features|       rawPrediction|         probability|prediction|
+----------------+------------+------------+----------------+-----------+----------+---------+---------------+---------------+------------+-----------+-----------+----+------+---------------+-----------------+-----------------+-----------------+--------------------+--------------------+--------------------+----------+
|             0.0|       -13.8|         2.2|            2000|126.9765873|37.5500359|39.869448|    575.6666667|    575.6666667| 8.3

In [20]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='등급', metricName='accuracy')

train_acc = evaluator.evaluate(train_pred)
test_acc = evaluator.evaluate(test_pred)

print('Train Accuracy:', train_acc)

Train Accuracy: 0.9002232929838994


In [21]:
print('Test Accuracy:', test_acc)

Test Accuracy: 0.9010494752623688


# Decision Tree

Test Accuracy : 85.8%

In [22]:
from pyspark.ml.classification import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(featuresCol='features', labelCol='등급', maxDepth=2)

dt_model = dt_clf.fit(train_ftr_vec)

train_pred = dt_model.transform(train_ftr_vec)
test_pred = dt_model.transform(test_ftr_vec)

In [23]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='등급', metricName='accuracy')

train_acc = evaluator.evaluate(train_pred)
test_acc = evaluator.evaluate(test_pred)

print('Train Accuracy:', train_acc)
print('Test Accuracy:', test_acc)

Train Accuracy: 0.8579151486661182
Test Accuracy: 0.8582526918359002


In [24]:
test_pred.show()

+----------------+------------+------------+----------------+-----------+----------+---------+---------------+---------------+------------+-----------+-----------+----+------+---------------+-----------------+-----------------+-----------------+--------------------+--------------------+--------------------+----------+
|현장안전센터거리|시간단위기온|시간단위풍속|시간단위가시거리|        lon|       lat| 실효습도|가스사용량(KWh)|전기사용량(KWh)|인명피해확률|   회귀예측|       지수|등급|인덱스|화재발생시각_밤|화재발생시각_새벽|화재발생시각_저녁|화재발생시각_주간|            features|       rawPrediction|         probability|prediction|
+----------------+------------+------------+----------------+-----------+----------+---------+---------------+---------------+------------+-----------+-----------+----+------+---------------+-----------------+-----------------+-----------------+--------------------+--------------------+--------------------+----------+
|             0.0|       -13.8|         2.2|            2000|126.9765873|37.5500359|39.869448|    575.6666667|    575.6666667| 8.3